### finish generating inter_gex/inter_atac

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
projdir = '/u/project/cluo/terencew/demux_benchmark/adelus_2023/'
donors = list(np.loadtxt(f'{projdir}/txt/donors.txt', dtype=str))
samples = list(np.loadtxt(f'{projdir}/txt/samples.txt', dtype=str))
s = samples[0]
clusters = 6

In [3]:
h_donors = list(np.loadtxt(f'{projdir}/txt/donors.txt', dtype=str))
mt13_donors = list(np.loadtxt(f'{projdir}/txt/MT1-3_donors.txt', dtype=str))
mt4_donors = list(np.loadtxt(f'{projdir}/txt/MT4_donors.txt', dtype=str))
mt5_donors = list(np.loadtxt(f'{projdir}/txt/MT5_donors.txt', dtype=str))

In [4]:
# samples = ['ctrl6h', 'il1b6h', 'pooled_endMT_1', 'pooled_endMT_2',
#        'pooled_endMT_3', 'pooled_endMT_4', 'pooled_endMT_5']
# samples = ['ctrl6h', 'il1b6h']

In [5]:
indir = f'{projdir}/demux/regular/demuxlet/gex/'
t = samples[0]
demuxlet_gex = pd.read_csv(f'{indir}/{t}.best', sep='\t', index_col=0)
demuxlet_gex['sample'] = [t] * demuxlet_gex.shape[0]

for t in samples[1:]:
    tmp = pd.read_csv(f'{indir}/{t}.best', sep='\t', index_col=0)
    tmp['sample'] = [t] * tmp.shape[0]
    demuxlet_gex = pd.concat([demuxlet_gex, tmp], axis=0)

demuxlet_gex['t'] = [x for x in demuxlet_gex['sample']]
demuxlet_gex['barcode'] = demuxlet_gex['BARCODE']
demuxlet_gex.index = [f'{x}_{y}' for x,y in zip(demuxlet_gex['barcode'],demuxlet_gex['t'])]

d_mask = demuxlet_gex['DROPLET.TYPE'] == 'DBL'
u_mask = demuxlet_gex['DROPLET.TYPE'] == 'AMB'
demuxlet_gex.loc[d_mask,'SNG.BEST.GUESS'] = ['doublet'] * sum(d_mask)
demuxlet_gex.loc[u_mask,'SNG.BEST.GUESS'] = ['unassigned'] * sum(u_mask)

demuxlet_gex.shape

(35201, 22)

In [6]:
indir = f'{projdir}/demux/regular/demuxlet/atac/'
t = samples[0]
demuxlet_atac = pd.read_csv(f'{indir}/{t}.best', sep='\t', index_col=0)
demuxlet_atac['sample'] = [t] * demuxlet_atac.shape[0]

for t in samples[1:]:
    tmp = pd.read_csv(f'{indir}/{t}.best', sep='\t', index_col=0)
    tmp['sample'] = [t] * tmp.shape[0]
    demuxlet_atac = pd.concat([demuxlet_atac, tmp], axis=0)

demuxlet_atac['t'] = [x for x in demuxlet_atac['sample']]
demuxlet_atac['barcode'] = demuxlet_atac['BARCODE']
demuxlet_atac.index = [f'{x}_{y}' for x,y in zip(demuxlet_atac['barcode'],demuxlet_atac['t'])]

d_mask = demuxlet_atac['DROPLET.TYPE'] == 'DBL'
u_mask = demuxlet_atac['DROPLET.TYPE'] == 'AMB'
demuxlet_atac.loc[d_mask,'SNG.BEST.GUESS'] = ['doublet'] * sum(d_mask)
demuxlet_atac.loc[u_mask,'SNG.BEST.GUESS'] = ['unassigned'] * sum(u_mask)

demuxlet_atac.shape

(35201, 22)

In [7]:
# for i in range(clusters):
#     barcodes = freemuxlet[freemuxlet['SNG.BEST.GUESS'] == f'{i}'].index
#     if barcodes.empty == False:
#         ### get best assigned donor, no unassigned/doublet
#         mask = [x in donors for x in demuxlet.loc[barcodes][]'SNG.BEST.GUESS']
#         maj_donor = demuxlet.loc[barcodes][mask]['SNG.BEST.GUESS'].mode().values[0]
#         freemuxlet['SNG.BEST.GUESS'].replace({f'{i}' : f'{maj_donor}'}, inplace=True)

In [8]:
def map_free_to_anchor(free, anchor, free_col, anchor_col, clusters, donors):
    for i in range(clusters):
        barcodes = free[free[free_col] == f'{i}'].index
        if barcodes.empty == False:
            ### get best assigned donor, no unassigned/doublet
            mask = [x in donors for x in anchor.loc[barcodes][anchor_col]]
            if not barcodes.empty:
                maj_donor = anchor.loc[barcodes][mask][anchor_col].mode().values[0]
            else:
                maj_donor = 'unassigned'
            free[free_col].replace({f'{i}' : f'{maj_donor}'}, inplace=True)
    return free

In [9]:
### because clusters are randomized, I need to assign freemuxlet at the individual level
demux_dir = f'{projdir}/demux/regular/demuxlet/gex/'
indir = f'{projdir}/demux/regular/freemuxlet/gex/'
t = samples[0]
clusters = 6
donors = h_donors.copy()

### 
tmp_demuxlet = pd.read_csv(f'{demux_dir}/{t}.best', sep='\t', index_col=0)
tmp_demuxlet.index = tmp_demuxlet['BARCODE']
tmp_demuxlet.sort_index(inplace=True)

freemuxlet_gex = pd.read_csv(f'{indir}/{t}.clust1.samples.gz', sep='\t', index_col=0)
freemuxlet_gex.index = freemuxlet_gex['BARCODE']
freemuxlet_gex['sample'] = [t] * freemuxlet_gex.shape[0]
freemuxlet_gex['SNG.BEST.GUESS'] = [str(x) for x in freemuxlet_gex['SNG.BEST.GUESS']]
freemuxlet_gex.sort_index(inplace=True)

freemuxlet_gex = map_free_to_anchor(freemuxlet_gex, tmp_demuxlet,
                                    'SNG.BEST.GUESS', 'SNG.BEST.GUESS',
                                    clusters, donors)
for t in samples[1:]:
    if t in ['pooled_endMT_1', 'pooled_endMT_2', 'pooled_endMT_3']:
        clusters = 6
        donors = mt13_donors.copy()
    elif t == 'pooled_endMT_4':
        clusters = 6
        donors = mt4_donors.copy()
    elif t == 'pooled_endMT_5':
        clusters = 5
        donors = mt5_donors.copy()
    tmp_demuxlet = pd.read_csv(f'{demux_dir}/{t}.best', sep='\t', index_col=0)
    tmp_demuxlet.index = tmp_demuxlet['BARCODE']
    tmp_demuxlet.sort_index(inplace=True)
    tmp_gex = pd.read_csv(f'{indir}/{t}.clust1.samples.gz', sep='\t', index_col=0)
    tmp_gex.index = tmp_gex['BARCODE']
    tmp_gex['sample'] = [t] * tmp_gex.shape[0]
    tmp_gex['SNG.BEST.GUESS'] = [str(x) for x in tmp_gex['SNG.BEST.GUESS']]
    tmp_gex.sort_index(inplace=True)
    tmp_gex = map_free_to_anchor(tmp_gex, tmp_demuxlet,
                                 'SNG.BEST.GUESS', 'SNG.BEST.GUESS',
                                 clusters, donors)
    freemuxlet_gex = pd.concat([freemuxlet_gex, tmp_gex], axis=0)

freemuxlet_gex['t'] = [x for x in freemuxlet_gex['sample']]
freemuxlet_gex['barcode'] = freemuxlet_gex['BARCODE']
freemuxlet_gex.index = [f'{x}_{y}' for x,y in zip(freemuxlet_gex['barcode'],freemuxlet_gex['t'])]

d_mask = freemuxlet_gex['DROPLET.TYPE'] == 'DBL'
u_mask = freemuxlet_gex['DROPLET.TYPE'] == 'AMB'
freemuxlet_gex.loc[d_mask,'SNG.BEST.GUESS'] = ['doublet'] * sum(d_mask)
freemuxlet_gex.loc[u_mask,'SNG.BEST.GUESS'] = ['unassigned'] * sum(u_mask)

freemuxlet_gex.shape

(35201, 22)

In [10]:
### because clusters are randomized, I need to assign freemuxlet at the individual level
demux_dir = f'{projdir}/demux/regular/demuxlet/atac/'
indir = f'{projdir}/demux/regular/freemuxlet/atac/'
t = samples[0]
clusters = 6
donors = h_donors.copy()

### 
tmp_demuxlet = pd.read_csv(f'{demux_dir}/{t}.best', sep='\t', index_col=0)
tmp_demuxlet.index = tmp_demuxlet['BARCODE']
tmp_demuxlet.sort_index(inplace=True)

freemuxlet_atac = pd.read_csv(f'{indir}/{t}.clust1.samples.gz', sep='\t', index_col=0)
freemuxlet_atac.index = freemuxlet_atac['BARCODE']
freemuxlet_atac['sample'] = [t] * freemuxlet_atac.shape[0]
freemuxlet_atac['SNG.BEST.GUESS'] = [str(x) for x in freemuxlet_atac['SNG.BEST.GUESS']]
freemuxlet_atac.sort_index(inplace=True)

freemuxlet_atac = map_free_to_anchor(freemuxlet_atac, tmp_demuxlet,
                                    'SNG.BEST.GUESS', 'SNG.BEST.GUESS',
                                    clusters, donors)
for t in samples[1:]:
    if t in ['pooled_endMT_1', 'pooled_endMT_2', 'pooled_endMT_3']:
        clusters = 6
        donors = mt13_donors.copy()
    elif t == 'pooled_endMT_4':
        clusters = 6
        donors = mt4_donors.copy()
    elif t == 'pooled_endMT_5':
        clusters = 5
        donors = mt5_donors.copy()
    tmp_demuxlet = pd.read_csv(f'{demux_dir}/{t}.best', sep='\t', index_col=0)
    tmp_demuxlet.index = tmp_demuxlet['BARCODE']
    tmp_demuxlet.sort_index(inplace=True)
    tmp_atac = pd.read_csv(f'{indir}/{t}.clust1.samples.gz', sep='\t', index_col=0)
    tmp_atac.index = tmp_atac['BARCODE']
    tmp_atac['sample'] = [t] * tmp_atac.shape[0]
    tmp_atac['SNG.BEST.GUESS'] = [str(x) for x in tmp_atac['SNG.BEST.GUESS']]
    tmp_atac.sort_index(inplace=True)
    tmp_atac = map_free_to_anchor(tmp_atac, tmp_demuxlet,
                                 'SNG.BEST.GUESS', 'SNG.BEST.GUESS',
                                 clusters, donors)
    freemuxlet_atac = pd.concat([freemuxlet_atac, tmp_atac], axis=0)

freemuxlet_atac['t'] = [x for x in freemuxlet_atac['sample']]
freemuxlet_atac['barcode'] = freemuxlet_atac['BARCODE']
freemuxlet_atac.index = [f'{x}_{y}' for x,y in zip(freemuxlet_atac['barcode'],freemuxlet_atac['t'])]

d_mask = freemuxlet_atac['DROPLET.TYPE'] == 'DBL'
u_mask = freemuxlet_atac['DROPLET.TYPE'] == 'AMB'
freemuxlet_atac.loc[d_mask,'SNG.BEST.GUESS'] = ['doublet'] * sum(d_mask)
freemuxlet_atac.loc[u_mask,'SNG.BEST.GUESS'] = ['unassigned'] * sum(u_mask)

freemuxlet_atac.shape

(35201, 22)

In [11]:
indir = f'{projdir}/demux/regular/vireo/gex/'
t = samples[0]
vireo_gex = pd.read_csv(f'{indir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
vireo_gex['sample'] = [t] * vireo_gex.shape[0]

for t in samples[1:]:
    tmp = pd.read_csv(f'{indir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
    tmp['sample'] = [t] * tmp.shape[0]
    vireo_gex = pd.concat([vireo_gex, tmp], axis=0)

vireo_gex['t'] = [x for x in vireo_gex['sample']]
vireo_gex['barcode'] = vireo_gex.index
vireo_gex.index = [f'{x}_{y}' for x,y in zip(vireo_gex['barcode'],vireo_gex['t'])]

vireo_gex.shape

(35201, 10)

In [12]:
indir = f'{projdir}/demux/regular/vireo/atac/'
t = samples[0]
vireo_atac = pd.read_csv(f'{indir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
vireo_atac['sample'] = [t] * vireo_atac.shape[0]

for t in samples[1:]:
    tmp = pd.read_csv(f'{indir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
    tmp['sample'] = [t] * tmp.shape[0]
    vireo_atac = pd.concat([vireo_atac, tmp], axis=0)

vireo_atac['t'] = [x for x in vireo_atac['sample']]
vireo_atac['barcode'] = vireo_atac.index
vireo_atac.index = [f'{x}_{y}' for x,y in zip(vireo_atac['barcode'],vireo_atac['t'])]

vireo_atac.shape

(35201, 10)

In [13]:
### because clusters are randomized, I need to assign freemuxlet at the individual level
demux_dir = f'{projdir}/demux/regular/vireo/gex/'
indir = f'{projdir}/demux/regular/vireo_nogenos/gex/'
t = samples[0]
clusters = 6
donors = h_donors.copy()

### 
tmp_vireo = pd.read_csv(f'{demux_dir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
vireo_nogenos_gex = pd.read_csv(f'{indir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
vireo_nogenos_gex['sample'] = [t] * vireo_nogenos_gex.shape[0]
vireo_nogenos_gex['donor_id'] = [str(x) for x in vireo_nogenos_gex['donor_id']]
vireo_nogenos_gex['barcode'] = vireo_nogenos_gex.index
vireo_nogenos_gex['donor_id'] = [x.replace('donor', '') for x in vireo_nogenos_gex['donor_id']]

vireo_nogenos_gex = map_free_to_anchor(vireo_nogenos_gex, tmp_vireo,
                                    'donor_id', 'donor_id',
                                    clusters, donors)
for t in samples[1:]:
    if t in ['pooled_endMT_1', 'pooled_endMT_2', 'pooled_endMT_3']:
        clusters = 6
        donors = mt13_donors.copy()
    elif t == 'pooled_endMT_4':
        clusters = 6
        donors = mt4_donors.copy()
    elif t == 'pooled_endMT_5':
        clusters = 5
        donors = mt5_donors.copy()
    tmp_vireo = pd.read_csv(f'{demux_dir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
    tmp_gex = pd.read_csv(f'{indir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
    tmp_gex['sample'] = [t] * tmp_gex.shape[0]
    tmp_gex['donor_id'] = [str(x) for x in tmp_gex['donor_id']]
    tmp_gex['barcode'] = tmp_gex.index
    tmp_gex['donor_id'] = [x.replace('donor', '') for x in tmp_gex['donor_id']]

    tmp_gex = map_free_to_anchor(tmp_gex, tmp_vireo,
                                 'donor_id', 'donor_id',
                                 clusters, donors)
    vireo_nogenos_gex = pd.concat([vireo_nogenos_gex, tmp_gex], axis=0)

vireo_nogenos_gex['t'] = [x for x in vireo_nogenos_gex['sample']]
vireo_nogenos_gex.index = [f'{x}_{y}' for x,y in zip(vireo_nogenos_gex['barcode'],vireo_nogenos_gex['t'])]

vireo_nogenos_gex.shape

(35201, 10)

In [14]:
### because clusters are randomized, I need to assign freemuxlet at the individual level
demux_dir = f'{projdir}/demux/regular/vireo/atac/'
indir = f'{projdir}/demux/regular/vireo_nogenos/atac/'
t = samples[0]
clusters = 6
donors = h_donors.copy()

### 
tmp_vireo = pd.read_csv(f'{demux_dir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
vireo_nogenos_atac = pd.read_csv(f'{indir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
vireo_nogenos_atac['sample'] = [t] * vireo_nogenos_atac.shape[0]
vireo_nogenos_atac['donor_id'] = [str(x) for x in vireo_nogenos_atac['donor_id']]
vireo_nogenos_atac['barcode'] = vireo_nogenos_atac.index
vireo_nogenos_atac['donor_id'] = [x.replace('donor', '') for x in vireo_nogenos_atac['donor_id']]

vireo_nogenos_atac = map_free_to_anchor(vireo_nogenos_atac, tmp_vireo,
                                    'donor_id', 'donor_id',
                                    clusters, donors)
for t in samples[1:]:
    if t in ['pooled_endMT_1', 'pooled_endMT_2', 'pooled_endMT_3']:
        clusters = 6
        donors = mt13_donors.copy()
    elif t == 'pooled_endMT_4':
        clusters = 6
        donors = mt4_donors.copy()
    elif t == 'pooled_endMT_5':
        clusters = 5
        donors = mt5_donors.copy()
    tmp_vireo = pd.read_csv(f'{demux_dir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
    tmp_atac = pd.read_csv(f'{indir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
    tmp_atac['sample'] = [t] * tmp_atac.shape[0]
    tmp_atac['donor_id'] = [str(x) for x in tmp_atac['donor_id']]
    tmp_atac['barcode'] = tmp_atac.index
    tmp_atac['donor_id'] = [x.replace('donor', '') for x in tmp_atac['donor_id']]

    tmp_atac = map_free_to_anchor(tmp_atac, tmp_vireo,
                                 'donor_id', 'donor_id',
                                 clusters, donors)
    vireo_nogenos_atac = pd.concat([vireo_nogenos_atac, tmp_atac], axis=0)

vireo_nogenos_atac['t'] = [x for x in vireo_nogenos_atac['sample']]
vireo_nogenos_atac.index = [f'{x}_{y}' for x,y in zip(vireo_nogenos_atac['barcode'],vireo_nogenos_atac['t'])]

vireo_nogenos_atac.shape

(35201, 10)

In [15]:
### because clusters are randomized, I need to assign souporcell at the individual level
demux_dir = f'{projdir}/demux/regular/vireo/gex/'
indir = f'{projdir}/demux/regular/souporcell/gex/'
t = samples[0]
clusters = 6
donors = h_donors.copy()

### 
tmp_vireo = pd.read_csv(f'{demux_dir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
souporcell_gex = pd.read_csv(f'{indir}/{t}/clusters.tsv', sep='\t', index_col=0)
souporcell_gex['sample'] = [t] * souporcell_gex.shape[0]
souporcell_gex['barcode'] = souporcell_gex.index

souporcell_gex = map_free_to_anchor(souporcell_gex, tmp_vireo,
                                    'assignment', 'donor_id',
                                    clusters, donors)
for t in samples[1:]:
    if t in ['pooled_endMT_1', 'pooled_endMT_2', 'pooled_endMT_3']:
        clusters = 6
        donors = mt13_donors.copy()
    elif t == 'pooled_endMT_4':
        clusters = 6
        donors = mt4_donors.copy()
    elif t == 'pooled_endMT_5':
        clusters = 5
        donors = mt5_donors.copy()
    tmp_vireo = pd.read_csv(f'{demux_dir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
    tmp_gex = pd.read_csv(f'{indir}/{t}/clusters.tsv', sep='\t', index_col=0)
    tmp_gex['sample'] = [t] * tmp_gex.shape[0]
    tmp_gex['barcode'] = tmp_gex.index
    tmp_gex = map_free_to_anchor(tmp_gex, tmp_vireo,
                                 'assignment', 'donor_id',
                                 clusters, donors)
    souporcell_gex = pd.concat([souporcell_gex, tmp_gex], axis=0)

souporcell_gex['t'] = [x for x in souporcell_gex['sample']]
souporcell_gex.index = [f'{x}_{y}' for x,y in zip(souporcell_gex['barcode'],souporcell_gex['t'])]

d_mask = souporcell_gex['status'] == 'doublet'
u_mask = souporcell_gex['status'] == 'unassigned'
souporcell_gex.loc[d_mask,'assignment'] = ['doublet'] * sum(d_mask)
souporcell_gex.loc[u_mask,'assignment'] = ['unassigned'] * sum(u_mask)

souporcell_gex.shape

(35201, 13)

In [16]:
### because clusters are randomized, I need to assign souporcell at the individual level
demux_dir = f'{projdir}/demux/regular/vireo/atac/'
indir = f'{projdir}/demux/regular/souporcell/atac/'
t = samples[0]
clusters = 6
donors = h_donors.copy()

### 
tmp_vireo = pd.read_csv(f'{demux_dir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
souporcell_atac = pd.read_csv(f'{indir}/{t}/clusters.tsv', sep='\t', index_col=0)
souporcell_atac['sample'] = [t] * souporcell_atac.shape[0]
souporcell_atac['barcode'] = souporcell_atac.index

souporcell_atac = map_free_to_anchor(souporcell_atac, tmp_vireo,
                                    'assignment', 'donor_id',
                                    clusters, donors)
for t in samples[1:]:
    if t in ['pooled_endMT_1', 'pooled_endMT_2', 'pooled_endMT_3']:
        clusters = 6
        donors = mt13_donors.copy()
    elif t == 'pooled_endMT_4':
        clusters = 6
        donors = mt4_donors.copy()
    elif t == 'pooled_endMT_5':
        clusters = 5
        donors = mt5_donors.copy()
    tmp_vireo = pd.read_csv(f'{demux_dir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
    tmp_atac = pd.read_csv(f'{indir}/{t}/clusters.tsv', sep='\t', index_col=0)
    tmp_atac['sample'] = [t] * tmp_atac.shape[0]
    tmp_atac['barcode'] = tmp_atac.index
    tmp_atac = map_free_to_anchor(tmp_atac, tmp_vireo,
                                 'assignment', 'donor_id',
                                 clusters, donors)
    souporcell_atac = pd.concat([souporcell_atac, tmp_atac], axis=0)

souporcell_atac['t'] = [x for x in souporcell_atac['sample']]
souporcell_atac.index = [f'{x}_{y}' for x,y in zip(souporcell_atac['barcode'],souporcell_atac['t'])]

d_mask = souporcell_atac['status'] == 'doublet'
u_mask = souporcell_atac['status'] == 'unassigned'
souporcell_atac.loc[d_mask,'assignment'] = ['doublet'] * sum(d_mask)
souporcell_atac.loc[u_mask,'assignment'] = ['unassigned'] * sum(u_mask)

souporcell_atac.shape

(35201, 13)

In [17]:
### because clusters are randomized, I need to assign souporcell_nogenos at the individual level
demux_dir = f'{projdir}/demux/regular/vireo/gex/'
indir = f'{projdir}/demux/regular/souporcell_nogenos/gex/'
t = samples[0]
clusters = 6
donors = h_donors.copy()

### 
tmp_vireo = pd.read_csv(f'{demux_dir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
souporcell_nogenos_gex = pd.read_csv(f'{indir}/{t}/clusters.tsv', sep='\t', index_col=0)
souporcell_nogenos_gex['sample'] = [t] * souporcell_nogenos_gex.shape[0]
souporcell_nogenos_gex['barcode'] = souporcell_nogenos_gex.index

souporcell_nogenos_gex = map_free_to_anchor(souporcell_nogenos_gex, tmp_vireo,
                                    'assignment', 'donor_id',
                                    clusters, donors)
for t in samples[1:]:
    if t in ['pooled_endMT_1', 'pooled_endMT_2', 'pooled_endMT_3']:
        clusters = 6
        donors = mt13_donors.copy()
    elif t == 'pooled_endMT_4':
        clusters = 6
        donors = mt4_donors.copy()
    elif t == 'pooled_endMT_5':
        clusters = 5
        donors = mt5_donors.copy()
    tmp_vireo = pd.read_csv(f'{demux_dir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
    tmp_gex = pd.read_csv(f'{indir}/{t}/clusters.tsv', sep='\t', index_col=0)
    tmp_gex['sample'] = [t] * tmp_gex.shape[0]
    tmp_gex['barcode'] = tmp_gex.index
    tmp_gex = map_free_to_anchor(tmp_gex, tmp_vireo,
                                 'assignment', 'donor_id',
                                 clusters, donors)
    souporcell_nogenos_gex = pd.concat([souporcell_nogenos_gex, tmp_gex], axis=0)

souporcell_nogenos_gex['t'] = [x for x in souporcell_nogenos_gex['sample']]
souporcell_nogenos_gex.index = [f'{x}_{y}' for x,y in zip(souporcell_nogenos_gex['barcode'],souporcell_nogenos_gex['t'])]

d_mask = souporcell_nogenos_gex['status'] == 'doublet'
u_mask = souporcell_nogenos_gex['status'] == 'unassigned'
souporcell_nogenos_gex.loc[d_mask,'assignment'] = ['doublet'] * sum(d_mask)
souporcell_nogenos_gex.loc[u_mask,'assignment'] = ['unassigned'] * sum(u_mask)

souporcell_nogenos_gex.shape

(35201, 13)

In [18]:
### because clusters are randomized, I need to assign souporcell_nogenos at the individual level
demux_dir = f'{projdir}/demux/regular/vireo/atac/'
indir = f'{projdir}/demux/regular/souporcell_nogenos/atac/'
t = samples[0]
clusters = 6
donors = h_donors.copy()

### 
tmp_vireo = pd.read_csv(f'{demux_dir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
souporcell_nogenos_atac = pd.read_csv(f'{indir}/{t}/clusters.tsv', sep='\t', index_col=0)
souporcell_nogenos_atac['sample'] = [t] * souporcell_nogenos_atac.shape[0]
souporcell_nogenos_atac['barcode'] = souporcell_nogenos_atac.index

souporcell_nogenos_atac = map_free_to_anchor(souporcell_nogenos_atac, tmp_vireo,
                                    'assignment', 'donor_id',
                                    clusters, donors)
for t in samples[1:]:
    if t in ['pooled_endMT_1', 'pooled_endMT_2', 'pooled_endMT_3']:
        clusters = 6
        donors = mt13_donors.copy()
    elif t == 'pooled_endMT_4':
        clusters = 6
        donors = mt4_donors.copy()
    elif t == 'pooled_endMT_5':
        clusters = 5
        donors = mt5_donors.copy()
    tmp_vireo = pd.read_csv(f'{demux_dir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
    tmp_atac = pd.read_csv(f'{indir}/{t}/clusters.tsv', sep='\t', index_col=0)
    tmp_atac['sample'] = [t] * tmp_atac.shape[0]
    tmp_atac['barcode'] = tmp_atac.index
    tmp_atac = map_free_to_anchor(tmp_atac, tmp_vireo,
                                 'assignment', 'donor_id',
                                 clusters, donors)
    souporcell_nogenos_atac = pd.concat([souporcell_nogenos_atac, tmp_atac], axis=0)

souporcell_nogenos_atac['t'] = [x for x in souporcell_nogenos_atac['sample']]
souporcell_nogenos_atac.index = [f'{x}_{y}' for x,y in zip(souporcell_nogenos_atac['barcode'],souporcell_nogenos_atac['t'])]

d_mask = souporcell_nogenos_atac['status'] == 'doublet'
u_mask = souporcell_nogenos_atac['status'] == 'unassigned'
souporcell_nogenos_atac.loc[d_mask, 'assignment'] = ['doublet'] * sum(d_mask)
souporcell_nogenos_atac.loc[u_mask, 'assignment'] = ['unassigned'] * sum(u_mask)

souporcell_nogenos_atac.shape

(35201, 13)

In [19]:
demux_dir = f'{projdir}/demux/regular/vireo/gex/'
indir = f'{projdir}/demux/regular/scsplit/gex/'
t = samples[0]
clusters = 6
donors = h_donors.copy()

### 
tmp_vireo = pd.read_csv(f'{demux_dir}/{t}/donor_ids.tsv', sep='\t', index_col=0)

barcodes = pd.read_csv(f'{indir}/{t}/out/scSplit_P_s_c.csv', sep=',', header=0, index_col=0).index
scsplit_gex = pd.DataFrame(index=barcodes)
scsplit_gex['donor_id'] = 'unassigned'

results = pd.read_csv(f'{indir}/{t}/out/scSplit_result.csv', comment='#', sep='\t', index_col=0)
b_mask = scsplit_gex.index.isin(results.index)
scsplit_gex.loc[b_mask,'donor_id'] = results['Cluster']
scsplit_gex['donor_id'] = [str(x).replace('SNG-', '') for x in scsplit_gex['donor_id']]
d_mask = ['DBL' in x for x in scsplit_gex['donor_id']]
scsplit_gex.loc[d_mask,'donor_id'] = 'doublet'
scsplit_gex = map_free_to_anchor(scsplit_gex, tmp_vireo, 'donor_id', 'donor_id', clusters+1, donors)
scsplit_gex['sample'] = t

for t in samples[1:]:
    if t in ['pooled_endMT_1', 'pooled_endMT_2', 'pooled_endMT_3']:
        clusters = 6
        donors = mt13_donors.copy()
    elif t == 'pooled_endMT_4':
        clusters = 6
        donors = mt4_donors.copy()
    elif t == 'pooled_endMT_5':
        clusters = 5
        donors = mt5_donors.copy()
    tmp_vireo = pd.read_csv(f'{demux_dir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
    
    barcodes = pd.read_csv(f'{indir}/{t}/out/scSplit_P_s_c.csv', sep=',', header=0, index_col=0).index
    tmp_gex = pd.DataFrame(index=barcodes)
    tmp_gex['donor_id'] = 'unassigned'

    results = pd.read_csv(f'{indir}/{t}/out/scSplit_result.csv', comment='#', sep='\t', index_col=0)
    b_mask = tmp_gex.index.isin(results.index)
    tmp_gex.loc[b_mask,'donor_id'] = results['Cluster']
    tmp_gex['donor_id'] = [str(x).replace('SNG-', '') for x in tmp_gex['donor_id']]
    d_mask = ['DBL' in x for x in tmp_gex['donor_id']]
    tmp_gex.loc[d_mask,'donor_id'] = 'doublet'
    tmp_gex = map_free_to_anchor(tmp_gex, tmp_vireo, 'donor_id', 'donor_id', clusters+1, donors)
    tmp_gex['sample'] = t
    scsplit_gex = pd.concat([scsplit_gex, tmp_gex], axis=0)

scsplit_gex['t'] = [x for x in scsplit_gex['sample']]
scsplit_gex.index = [f'{x}_{y}' for x,y in zip(scsplit_gex.index,scsplit_gex['t'])]
    
scsplit_gex.shape

(35201, 3)

In [20]:
demux_dir = f'{projdir}/demux/regular/vireo/atac/'
indir = f'{projdir}/demux/regular/scsplit/atac/'
t = samples[0]
clusters = 6
donors = h_donors.copy()

### 
tmp_vireo = pd.read_csv(f'{demux_dir}/{t}/donor_ids.tsv', sep='\t', index_col=0)

barcodes = pd.read_csv(f'{indir}/{t}/out/scSplit_P_s_c.csv', sep=',', header=0, index_col=0).index
scsplit_atac = pd.DataFrame(index=barcodes)
scsplit_atac['donor_id'] = 'unassigned'

results = pd.read_csv(f'{indir}/{t}/out/scSplit_result.csv', comment='#', sep='\t', index_col=0)
b_mask = scsplit_atac.index.isin(results.index)
scsplit_atac.loc[b_mask,'donor_id'] = results['Cluster']
scsplit_atac['donor_id'] = [str(x).replace('SNG-', '') for x in scsplit_atac['donor_id']]
d_mask = ['DBL' in x for x in scsplit_atac['donor_id']]
scsplit_atac.loc[d_mask,'donor_id'] = 'doublet'
scsplit_atac = map_free_to_anchor(scsplit_atac, tmp_vireo, 'donor_id', 'donor_id', clusters+1, donors)
scsplit_atac['sample'] = t

for t in samples[1:]:
    if t in ['pooled_endMT_1', 'pooled_endMT_2', 'pooled_endMT_3']:
        clusters = 6
        donors = mt13_donors.copy()
    elif t == 'pooled_endMT_4':
        clusters = 6
        donors = mt4_donors.copy()
    elif t == 'pooled_endMT_5':
        clusters = 5
        donors = mt5_donors.copy()
    tmp_vireo = pd.read_csv(f'{demux_dir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
    
    barcodes = pd.read_csv(f'{indir}/{t}/out/scSplit_P_s_c.csv', sep=',', header=0, index_col=0).index
    tmp_atac = pd.DataFrame(index=barcodes)
    tmp_atac['donor_id'] = 'unassigned'

    results = pd.read_csv(f'{indir}/{t}/out/scSplit_result.csv', comment='#', sep='\t', index_col=0)
    b_mask = tmp_atac.index.isin(results.index)
    tmp_atac.loc[b_mask,'donor_id'] = results['Cluster']
    tmp_atac['donor_id'] = [str(x).replace('SNG-', '') for x in tmp_atac['donor_id']]
    d_mask = ['DBL' in x for x in tmp_atac['donor_id']]
    tmp_atac.loc[d_mask,'donor_id'] = 'doublet'
    tmp_atac = map_free_to_anchor(tmp_atac, tmp_vireo, 'donor_id', 'donor_id', clusters+1, donors)
    tmp_atac['sample'] = t
    scsplit_atac = pd.concat([scsplit_atac, tmp_atac], axis=0)
    
scsplit_atac['t'] = [x for x in scsplit_atac['sample']]
scsplit_atac.index = [f'{x}_{y}' for x,y in zip(scsplit_atac.index,scsplit_atac['t'])]
scsplit_atac.shape

(35201, 3)

In [21]:
demux_dir = f'{projdir}/demux/regular/vireo/gex/'
indir = f'{projdir}/demux/regular/scsplit_nogenos/gex/'
t = samples[0]
clusters = 6
donors = h_donors.copy()

### 
tmp_vireo = pd.read_csv(f'{demux_dir}/{t}/donor_ids.tsv', sep='\t', index_col=0)

barcodes = pd.read_csv(f'{indir}/{t}/scSplit_P_s_c.csv', sep=',', header=0, index_col=0).index
scsplit_nogenos_gex = pd.DataFrame(index=barcodes)
scsplit_nogenos_gex['donor_id'] = 'unassigned'

results = pd.read_csv(f'{indir}/{t}/scSplit_result.csv', comment='#', sep='\t', index_col=0)
b_mask = scsplit_nogenos_gex.index.isin(results.index)
scsplit_nogenos_gex.loc[b_mask,'donor_id'] = results['Cluster']
scsplit_nogenos_gex['donor_id'] = [str(x).replace('SNG-', '') for x in scsplit_nogenos_gex['donor_id']]
d_mask = ['DBL' in x for x in scsplit_nogenos_gex['donor_id']]
scsplit_nogenos_gex.loc[d_mask,'donor_id'] = 'doublet'
scsplit_nogenos_gex = map_free_to_anchor(scsplit_nogenos_gex, tmp_vireo, 'donor_id', 'donor_id', clusters+1, donors)
scsplit_nogenos_gex['sample'] = t

for t in samples[1:]:
    if t in ['pooled_endMT_1', 'pooled_endMT_2', 'pooled_endMT_3']:
        clusters = 6
        donors = mt13_donors.copy()
    elif t == 'pooled_endMT_4':
        clusters = 6
        donors = mt4_donors.copy()
    elif t == 'pooled_endMT_5':
        clusters = 5
        donors = mt5_donors.copy()
    tmp_vireo = pd.read_csv(f'{demux_dir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
    
    barcodes = pd.read_csv(f'{indir}/{t}/scSplit_P_s_c.csv', sep=',', header=0, index_col=0).index
    tmp_gex = pd.DataFrame(index=barcodes)
    tmp_gex['donor_id'] = 'unassigned'

    results = pd.read_csv(f'{indir}/{t}/scSplit_result.csv', comment='#', sep='\t', index_col=0)
    b_mask = tmp_gex.index.isin(results.index)
    tmp_gex.loc[b_mask,'donor_id'] = results['Cluster']
    tmp_gex['donor_id'] = [str(x).replace('SNG-', '') for x in tmp_gex['donor_id']]
    d_mask = ['DBL' in x for x in tmp_gex['donor_id']]
    tmp_gex.loc[d_mask,'donor_id'] = 'doublet'
    tmp_gex = map_free_to_anchor(tmp_gex, tmp_vireo, 'donor_id', 'donor_id', clusters+1, donors)
    tmp_gex['sample'] = t
    scsplit_nogenos_gex = pd.concat([scsplit_nogenos_gex, tmp_gex], axis=0)

scsplit_nogenos_gex['t'] = [x for x in scsplit_nogenos_gex['sample']]
scsplit_nogenos_gex.index = [f'{x}_{y}' for x,y in zip(scsplit_nogenos_gex.index,scsplit_nogenos_gex['t'])]
    
scsplit_nogenos_gex.shape

(35201, 3)

In [22]:
demux_dir = f'{projdir}/demux/regular/vireo/atac/'
indir = f'{projdir}/demux/regular/scsplit_nogenos/atac/'
t = samples[0]
clusters = 6
donors = h_donors.copy()

### 
tmp_vireo = pd.read_csv(f'{demux_dir}/{t}/donor_ids.tsv', sep='\t', index_col=0)

barcodes = pd.read_csv(f'{indir}/{t}/scSplit_P_s_c.csv', sep=',', header=0, index_col=0).index
scsplit_nogenos_atac = pd.DataFrame(index=barcodes)
scsplit_nogenos_atac['donor_id'] = 'unassigned'

results = pd.read_csv(f'{indir}/{t}/scSplit_result.csv', comment='#', sep='\t', index_col=0)
b_mask = scsplit_nogenos_atac.index.isin(results.index)
scsplit_nogenos_atac.loc[b_mask,'donor_id'] = results['Cluster']
scsplit_nogenos_atac['donor_id'] = [str(x).replace('SNG-', '') for x in scsplit_nogenos_atac['donor_id']]
d_mask = ['DBL' in x for x in scsplit_nogenos_atac['donor_id']]
scsplit_nogenos_atac.loc[d_mask,'donor_id'] = 'doublet'
scsplit_nogenos_atac = map_free_to_anchor(scsplit_nogenos_atac, tmp_vireo, 'donor_id', 'donor_id', clusters+1, donors)
scsplit_nogenos_atac['sample'] = t

for t in samples[1:]:
    if t in ['pooled_endMT_1', 'pooled_endMT_2', 'pooled_endMT_3']:
        clusters = 6
        donors = mt13_donors.copy()
    elif t == 'pooled_endMT_4':
        clusters = 6
        donors = mt4_donors.copy()
    elif t == 'pooled_endMT_5':
        clusters = 5
        donors = mt5_donors.copy()
    tmp_vireo = pd.read_csv(f'{demux_dir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
    
    barcodes = pd.read_csv(f'{indir}/{t}/scSplit_P_s_c.csv', sep=',', header=0, index_col=0).index
    tmp_atac = pd.DataFrame(index=barcodes)
    tmp_atac['donor_id'] = 'unassigned'

    results = pd.read_csv(f'{indir}/{t}/scSplit_result.csv', comment='#', sep='\t', index_col=0)
    b_mask = tmp_atac.index.isin(results.index)
    tmp_atac.loc[b_mask,'donor_id'] = results['Cluster']
    tmp_atac['donor_id'] = [str(x).replace('SNG-', '') for x in tmp_atac['donor_id']]
    d_mask = ['DBL' in x for x in tmp_atac['donor_id']]
    tmp_atac.loc[d_mask,'donor_id'] = 'doublet'
    tmp_atac = map_free_to_anchor(tmp_atac, tmp_vireo, 'donor_id', 'donor_id', clusters+1, donors)
    tmp_atac['sample'] = t
    scsplit_nogenos_atac = pd.concat([scsplit_nogenos_atac, tmp_atac], axis=0)

scsplit_nogenos_atac['t'] = [x for x in scsplit_nogenos_atac['sample']]
scsplit_nogenos_atac.index = [f'{x}_{y}' for x,y in zip(scsplit_nogenos_atac.index,scsplit_nogenos_atac['t'])]
    
scsplit_nogenos_atac.shape

(35201, 3)

In [23]:
indir = f'{projdir}/demux/regular/demuxalot/gex'
t = samples[0]
demuxalot_gex = pd.read_csv(f'{indir}/{t}/{t}_posteriors.csv', comment='#', sep='\t', index_col=0)
demuxalot_gex[f'assignment'] = demuxalot_gex.idxmax(axis=1)
demuxalot_gex['doublet_status'] = ['+' in x for x in demuxalot_gex[f'assignment']]
mask = demuxalot_gex['doublet_status']
demuxalot_gex.loc[mask,f'assignment'] = 'doublet'
demuxalot_gex['sample'] = [t] * demuxalot_gex.shape[0]

for t in samples[1:]:
    tmp = pd.read_csv(f'{indir}/{t}/{t}_posteriors.csv', comment='#', sep='\t', index_col=0)
    tmp[f'assignment'] = tmp.idxmax(axis=1)
    tmp['doublet_status'] = ['+' in x for x in tmp[f'assignment']]
    mask = tmp['doublet_status']
    tmp.loc[mask,f'assignment'] = 'doublet'
    tmp['sample'] = [t] * tmp.shape[0]
    demuxalot_gex = pd.concat([demuxalot_gex, tmp], axis=0)

demuxalot_gex['barcode'] = demuxalot_gex.index
demuxalot_gex.index = [f'{x}_{y}' for x,y in zip(demuxalot_gex['barcode'],demuxalot_gex['sample'])]
    
demuxalot_gex.shape    

(35201, 52)

In [24]:
vireo_dir = f'{projdir}/demux/regular/vireo/atac/'
indir = f'{projdir}/demux/regular/scavengers/atac/'

t = samples[0]
clusters = 6
donors = h_donors.copy()

tmp_vireo = pd.read_csv(f'{vireo_dir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
scavengers_atac = pd.read_csv(f'{indir}/{t}/clusters.final.tsv', sep='\t')
scavengers_atac['assignment'] = scavengers_atac['barcode']
scavengers_atac['barcode'] = [x[0] for x in scavengers_atac.index]
scavengers_atac['status'] = [x[1] for x in scavengers_atac.index]
scavengers_atac.index = scavengers_atac['barcode']

d_mask = scavengers_atac['status'] == 'doublet'
u_mask = scavengers_atac['status'] == 'unassigned'
scavengers_atac.loc[d_mask,'assignment'] = ['doublet'] * sum(d_mask)
scavengers_atac.loc[u_mask,'assignment'] = ['unassigned'] * sum(u_mask)
scavengers_atac['sample'] = t
scavengers_atac = map_free_to_anchor(scavengers_atac, tmp_vireo, 'assignment', 'donor_id', clusters, donors)

for t in samples[1:]:
    print(t)
    if t in ['pooled_endMT_1', 'pooled_endMT_2', 'pooled_endMT_3']:
        clusters = 6
        donors = mt13_donors.copy()
    elif t == 'pooled_endMT_4':
        clusters = 6
        donors = mt4_donors.copy()
    elif t == 'pooled_endMT_5':
        clusters = 5
        donors = mt5_donors.copy()
    else:
        clusters = 6
        donors = h_donors.copy()
    tmp_vireo = pd.read_csv(f'{vireo_dir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
    tmp_atac = pd.read_csv(f'{indir}/{t}/clusters.final.tsv', sep='\t')
    tmp_atac['assignment'] = tmp_atac['barcode']
    tmp_atac['barcode'] = [x[0] for x in tmp_atac.index]
    tmp_atac['status'] = [x[1] for x in tmp_atac.index]
    tmp_atac.index = tmp_atac['barcode']

    d_mask = tmp_atac['status'] == 'doublet'
    u_mask = tmp_atac['status'] == 'unassigned'
    tmp_atac.loc[d_mask,'assignment'] = ['doublet'] * sum(d_mask)
    tmp_atac.loc[u_mask,'assignment'] = ['unassigned'] * sum(u_mask)
    tmp_atac = map_free_to_anchor(tmp_atac, tmp_vireo, 'assignment', 'donor_id', clusters, donors)
    tmp_atac['sample'] = t
    scavengers_atac = pd.concat([scavengers_atac, tmp_atac], axis=0)

scavengers_atac['t'] = [x for x in scavengers_atac['sample']]
scavengers_atac['barcode'] = scavengers_atac.index
scavengers_atac.index = [f'{x}_{y}' for x,y in zip(scavengers_atac['barcode'],scavengers_atac['t'])]

d_mask = scavengers_atac['status'] == 'doublet'
u_mask = scavengers_atac['status'] == 'unassigned'
scavengers_atac.loc[d_mask,'assignment'] = ['doublet'] * sum(d_mask)
scavengers_atac.loc[u_mask,'assignment'] = ['unassigned'] * sum(u_mask)

scavengers_atac.shape

il1b6h
pooled_endMT_1
pooled_endMT_2
pooled_endMT_3
pooled_endMT_4
pooled_endMT_5


(35201, 13)

In [25]:
demuxlet_gex.to_csv(f'{projdir}/csv/demux/gex/demuxlet.csv', sep='\t', header=True, index=True)
freemuxlet_gex.to_csv(f'{projdir}/csv/demux/gex/freemuxlet.csv', sep='\t', header=True, index=True)
vireo_gex.to_csv(f'{projdir}/csv/demux/gex/vireo.csv', sep='\t', header=True, index=True)
vireo_nogenos_gex.to_csv(f'{projdir}/csv/demux/gex/vireo_nogenos.csv', sep='\t', header=True, index=True)
souporcell_gex.to_csv(f'{projdir}/csv/demux/gex/souporcell.csv', sep='\t', header=True, index=True)
souporcell_nogenos_gex.to_csv(f'{projdir}/csv/demux/gex/souporcell_nogenos.csv', sep='\t', header=True, index=True)
demuxalot_gex.to_csv(f'{projdir}/csv/demux/gex/demuxalot.csv', sep='\t', header=True, index=True)
scsplit_gex.to_csv(f'{projdir}/csv/demux/gex/scsplit.csv', sep='\t', header=True, index=True)
scsplit_nogenos_gex.to_csv(f'{projdir}/csv/demux/gex/scsplit_nogenos.csv', sep='\t', header=True, index=True)

In [26]:
demuxlet_atac.to_csv(f'{projdir}/csv/demux/atac/demuxlet.csv', sep='\t', header=True, index=True)
freemuxlet_atac.to_csv(f'{projdir}/csv/demux/atac/freemuxlet.csv', sep='\t', header=True, index=True)
vireo_atac.to_csv(f'{projdir}/csv/demux/atac/vireo.csv', sep='\t', header=True, index=True)
vireo_nogenos_atac.to_csv(f'{projdir}/csv/demux/atac/vireo_nogenos.csv', sep='\t', header=True, index=True)
souporcell_atac.to_csv(f'{projdir}/csv/demux/atac/souporcell.csv', sep='\t', header=True, index=True)
souporcell_nogenos_atac.to_csv(f'{projdir}/csv/demux/atac/souporcell_nogenos.csv', sep='\t', header=True, index=True)
scsplit_atac.to_csv(f'{projdir}/csv/demux/atac/scsplit.csv', sep='\t', header=True, index=True)
scsplit_nogenos_atac.to_csv(f'{projdir}/csv/demux/atac/scsplit_nogenos.csv', sep='\t', header=True, index=True)
scavengers_atac.to_csv(f'{projdir}/csv/demux/atac/scavengers.csv', sep='\t', header=True, index=True)

In [27]:
scsplit_gex.sort_index(inplace=True)
scsplit_nogenos_gex.sort_index(inplace=True)
scsplit_atac.sort_index(inplace=True)
scsplit_nogenos_atac.sort_index(inplace=True)

In [28]:
inter_gex = pd.DataFrame(index=vireo_gex.index)
inter_gex['demuxlet'] = demuxlet_gex['SNG.BEST.GUESS']
inter_gex['freemuxlet'] = freemuxlet_gex['SNG.BEST.GUESS']
inter_gex['vireo'] = vireo_gex['donor_id']
inter_gex['vireo_nogenos'] = vireo_nogenos_gex['donor_id']
inter_gex['souporcell'] = souporcell_gex['assignment']
inter_gex['souporcell_nogenos'] = souporcell_nogenos_gex['assignment']
inter_gex['demuxalot'] = demuxalot_gex['assignment']
inter_gex['scsplit'] = scsplit_gex['donor_id']
inter_gex['scsplit_nogenos'] = scsplit_nogenos_gex['donor_id']

In [29]:
inter_atac = pd.DataFrame(index=vireo_atac.index)
inter_atac['demuxlet'] = demuxlet_atac['SNG.BEST.GUESS']
inter_atac['freemuxlet'] = freemuxlet_atac['SNG.BEST.GUESS']
inter_atac['vireo'] = vireo_atac['donor_id']
inter_atac['vireo_nogenos'] = vireo_nogenos_atac['donor_id']
inter_atac['souporcell'] = souporcell_atac['assignment']
inter_atac['souporcell_nogenos'] = souporcell_nogenos_atac['assignment']
inter_atac['scsplit'] = scsplit_atac['donor_id']
inter_atac['scsplit_nogenos'] = scsplit_nogenos_atac['donor_id']
inter_atac['scavengers'] = scavengers_atac['assignment']

In [30]:
inter_gex.to_csv(f'{projdir}/csv/demux/inter_gex_raw_donors.csv', sep='\t', header=True, index=True)
inter_atac.to_csv(f'{projdir}/csv/demux/inter_atac_raw_donors.csv', sep='\t', header=True, index=True)

In [31]:
### fix this now lmao

In [32]:
inter_gex['demuxlet'].value_counts().sort_index()

D120.rep0.uD562                     571
D120.rep0.uD562_D120.rep0.uD562     770
J76.rep2.uD52                       868
J98.rep0.uD9                       5120
J98.rep0.uD9_J98.rep0.uD9           911
M103.rep0.uD62                     1298
T100.rep0.uD20_T100.rep0.uD20      1054
TELOS                              4066
U114.rep0.uD563                    3423
U114.rep0.uD563_U114.rep0.uD563    1956
U115.rep0.uD557_U115.rep0.uD557     224
V101.rep0.uD22_V101.rep0.uD22      1632
V105.rep1.uD60                     3066
doublet                            6536
unassigned                         3706
Name: demuxlet, dtype: int64

In [33]:
replace_dict = {'U114.rep0.uD563_U114.rep0.uD563' : 'U114.rep0.uD563', 
                'D120.rep0.uD562_D120.rep0.uD562' : 'D120.rep0.uD562',
               'J98.rep0.uD9_J98.rep0.uD9' : 'J98.rep0.uD9'}
inter_gex.replace({''})

,demuxlet,freemuxlet,vireo,vireo_nogenos,souporcell,souporcell_nogenos,demuxalot,scsplit,scsplit_nogenos
AAACAGCCAACTAGCC-1_ctrl6h,U114.rep0.uD563_U114.rep0.uD563,U114.rep0.uD563_U114.rep0.uD563,U114.rep0.uD563_U114.rep0.uD563,U114.rep0.uD563_U114.rep0.uD563,U114.rep0.uD563_U114.rep0.uD563,U114.rep0.uD563_U114.rep0.uD563,U114.rep0.uD563_U114.rep0.uD563,U114.rep0.uD563_U114.rep0.uD563,U114.rep0.uD563_U114.rep0.uD563
AAACAGCCACACAATT-1_ctrl6h,D120.rep0.uD562_D120.rep0.uD562,D120.rep0.uD562_D120.rep0.uD562,D120.rep0.uD562_D120.rep0.uD562,D120.rep0.uD562_D120.rep0.uD562,D120.rep0.uD562_D120.rep0.uD562,D120.rep0.uD562_D120.rep0.uD562,D120.rep0.uD562_D120.rep0.uD562,D120.rep0.uD562_D120.rep0.uD562,D120.rep0.uD562_D120.rep0.uD562
AAACAGCCACTCGCTC-1_ctrl6h,U115.rep0.uD557_U115.rep0.uD557,U115.rep0.uD557_U115.rep0.uD557,U115.rep0.uD557_U115.rep0.uD557,U115.rep0.uD557_U115.rep0.uD557,U115.rep0.uD557_U115.rep0.uD557,U115.rep0.uD557_U115.rep0.uD557,U115.rep0.uD557_U115.rep0.uD557,doublet,U114.rep0.uD563_U114.rep0.uD563
AAACCAACAGAACCGA-1_ctrl6h,J98.rep0.uD9_J98.rep0.uD9,J98.rep0.uD9_J98.rep0.uD9,J98.rep0.uD9_J98.rep0.uD9,J98.rep0.uD9_J98.rep0.uD9,J98.rep0.uD9_J98.rep0.uD9,J98.rep0.uD9_J98.rep0.uD9,doublet,T100.rep0.uD20_T100.rep0.uD20,U114.rep0.uD563_U114.rep0.uD563
AAACCGAAGCTATGAC-1_ctrl6h,U114.rep0.uD563_U114.rep0.uD563,U114.rep0.uD563_U114.rep0.uD563,U114.rep0.uD563_U114.rep0.uD563,U114.rep0.uD563_U114.rep0.uD563,U114.rep0.uD563_U114.rep0.uD563,U114.rep0.uD563_U114.rep0.uD563,U114.rep0.uD563_U114.rep0.uD563,V101.rep0.uD22_V101.rep0.uD22,V101.rep0.uD22_V101.rep0.uD22
...,...,...,...,...,...,...,...,...,...
TTTGTGTTCGGGATTT-1_pooled_endMT_5,V105.rep1.uD60,V105.rep1.uD60,V105.rep1.uD60,V105.rep1.uD60,V105.rep1.uD60,V105.rep1.uD60,V105.rep1.uD60,V105.rep1.uD60,V105.rep1.uD60
TTTGTGTTCGTTATAG-1_pooled_endMT_5,doublet,TELOS,TELOS,TELOS,TELOS,TELOS,TELOS,U114.rep0.uD563,U114.rep0.uD563
TTTGTTGGTCCCGAAG-1_pooled_endMT_5,unassigned,TELOS,unassigned,TELOS,TELOS,TELOS,TELOS,TELOS,TELOS
TTTGTTGGTGAGACTC-1_pooled_endMT_5,U114.rep0.uD563,U114.rep0.uD563,U114.rep0.uD563,U114.rep0.uD563,U114.rep0.uD563,U114.rep0.uD563,U114.rep0.uD563,U114.rep0.uD563,U114.rep0.uD563


In [34]:
singlet_donors = sorted(inter_gex['demuxlet'].unique())[:-2]
for i, donor in enumerate(singlet_donors):
    inter_gex.replace({donor : f'donor{i}'}, inplace=True)
    inter_atac.replace({donor : f'donor{i}'}, inplace=True)

In [35]:
inter_gex.to_csv(f'{projdir}/csv/demux/inter_gex.csv', sep='\t', header=True, index=True)
inter_atac.to_csv(f'{projdir}/csv/demux/inter_atac.csv', sep='\t', header=True, index=True)